In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
from nltk.corpus import stopwords
import glob
from datetime import datetime
import statistics 
import pickle
import jmespath
import json
from rake_nltk import Rake
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from gensim.test.utils import datapath
import math
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

//anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
//anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict
//anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
forum = pd.read_csv('COPA_forum_replies.csv')
forum = forum[['ThreadReplyDate', 'UserID', 'FormattedBody']]
forum.head()

,ThreadReplyDate,UserID,FormattedBody
0,2008-06-25 10:57:03,3870,You are cordially invited to a Cirrus BBQ & Fu...
1,2005-10-23 00:56:48,4255,"Ed,Thank you for the offer. My envelope is on ..."
2,2005-10-23 02:02:14,3426,If I come over Tuesday do I get to play Igor t...
3,2005-10-24 04:31:25,2296,"Ed, My envelope is on the way also! thanks for..."
4,2005-11-04 01:34:09,3586,"When I returned home from S FL yesterday, I co..."


In [5]:
t1 = datetime.now()

data = forum.FormattedBody.values.tolist()

# Create a model for topic classification based on the dataset we have 
# (It may cause some problems because the model will be applied to the dataset later to give each content a topic.)
# (The problem can be solved when we get the entire dataset since we can take part of the contents as corpus.)

# Get some stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

t2 = datetime.now()
diff = t2 - t1
print (diff.seconds)

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

t3 = datetime.now()
diff = t3 - t2
print (diff.seconds)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

t4 = datetime.now()
diff = t4 - t3
print (diff.seconds)

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           passes=10)

pprint(lda_model.print_topics(num_words = 10))
# Get 20 related topics
doc_lda = lda_model[corpus]

t5 = datetime.now()
diff = t5 - t4
print (diff.seconds)

In [6]:
filename = 'id2word.pickle'
outfile = open(filename,'wb')
pickle.dump(id2word, outfile)
outfile.close()

In [7]:
filename = 'lda_model.pickle'
outfile = open(filename,'wb')
pickle.dump(lda_model, outfile)
outfile.close()